# Eat Safe, Love

## Notebook Set Up

In [1]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [4]:
# review the collections in our database
print(db.list_collection_names())

['establishments']


In [5]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [9]:
# Find the establishments with a hygiene score of 20
query = {"scores.Hygiene": 20}
req_fields = {"_id":1, "AddressLine1":1, "BusinessName":1, "BusinessType":1, "scores":1}

# Use count_documents to display the number of documents in the result
sort = [("scores.Hygiene", -1)]

# Display the first document in the results using pprint
print()
pprint(establishments.find(query, req_fields).sort(sort)[0])


{'AddressLine1': '5-6 Southfields Road',
 'BusinessName': 'The Chase Rest Home',
 'BusinessType': 'Caring Premises',
 '_id': ObjectId('679991ed773e8248a2c9b204'),
 'scores': {'ConfidenceInManagement': 20, 'Hygiene': 20, 'Structural': 20}}


In [10]:
# Convert the result to a Pandas DataFrame
hygiene_results = establishments.find(query, req_fields).sort(sort)
hygiene_results_df = pd.json_normalize(results)
# Display the number of rows in the DataFrame
print(hygiene_results_df.shape)
# Display the first 10 rows of the DataFrame
hygiene_results_df.head(10)


(0, 0)


""


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [11]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {"LocalAuthorityName": {"$regex": "London"},
         "RatingValue": {"$gte": 4}}
req_fields = {"_id":1, "AddressLine1":1, "BusinessName":1, "BusinessType":1, "scores":1}

# Use count_documents to display the number of documents in the result
print(establishments.count_documents(query))

# Display the first document in the results using pprint
print()
pprint(establishments.find(query, req_fields).sort(sort)[0])


33

{'AddressLine1': 'Air side London City Airport',
 'BusinessName': 'Brick Lane Brews',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 '_id': ObjectId('679991f1773e8248a2ca1470'),
 'scores': {'ConfidenceInManagement': 5, 'Hygiene': 10, 'Structural': 5}}


In [14]:
# Convert the result to a Pandas DataFrame
results = establishments.find(query, req_fields).sort(sort)
london_df = pd.json_normalize(results)
# Display the number of rows in the DataFrame
print(london_df.shape)
# Display the first 10 rows of the DataFrame
london_df.head(10)

(33, 7)


,_id,BusinessName,BusinessType,AddressLine1,scores.Hygiene,scores.Structural,scores.ConfidenceInManagement
0,679991f1773e8248a2ca1470,Brick Lane Brews,Restaurant/Cafe/Canteen,Air side London City Airport,10,5,5
1,679991ee773e8248a2c9cc1f,Charlie's,Other catering premises,Oak Apple Farm Building 103 Sheernes Docks,5,10,5
2,679991f1773e8248a2ca166b,Caffè Nero,Restaurant/Cafe/Canteen,London City Airport Airside Departure Lounge,5,5,0
3,679991f2773e8248a2ca190b,Mv Sunborn Yacht Hotel,Hotel/bed & breakfast/guest house,Western Gateway,5,5,5
4,679991f2773e8248a2ca1b16,Good Hotel London,Other catering premises,Western Gateway Royal Victoria Dock,5,5,5
5,679991ee773e8248a2c9cf45,Mv City Cruises Erasmus,Other catering premises,Cherry Garden Pier,0,5,0
6,679991ef773e8248a2c9da92,Benfleet Motor Yacht Club,Other catering premises,Ferry Road,0,0,10
7,679991ef773e8248a2c9e892,Coombs Catering t/a The Lock and Key,Restaurant/Cafe/Canteen,Leslie Ford House,0,5,0
8,679991ef773e8248a2c9e893,Tilbury Seafarers Centre,Restaurant/Cafe/Canteen,Tenants Row Tilbury Docks,0,0,0
9,679991f0773e8248a2c9ef9b,Mv Valulla,Other catering premises,Reeds River Cruises LtdKings ReachRiver Thames...,0,0,0


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [15]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.49014200
longitude = 0.08384000

query = {"geocode.latitude": {"$gte": latitude - degree_search, "$lte": latitude + degree_search},
        "geocode.longitude": {"$gte": longitude - degree_search, "$lte": longitude + degree_search},
        "RatingValue": 5}
req_fields = {"_id":1, "AddressLine1":1, "BusinessName":1, "BusinessType":1, "scores":1}
sort = [("scores.Hygiene", 1)]
limit = 5

# Print the results
# Use count_documents to display the number of documents in the result
print(establishments.count_documents(query))

# Display the first document in the results using pprint
print()
pprint(establishments.find(query, req_fields).sort(sort)[0])

87

{'AddressLine1': '144 - 146 Plumstead High Street',
 'BusinessName': 'Iceland',
 'BusinessType': 'Retailers - supermarkets/hypermarkets',
 '_id': ObjectId('679991f1773e8248a2ca0a36'),
 'scores': {'ConfidenceInManagement': 5, 'Hygiene': 0, 'Structural': 5}}


In [21]:
# Convert the result to a Pandas DataFrame
results = establishments.find(query, req_fields).sort(sort)
top_df = pd.json_normalize(results)
# Display the number of rows in the DataFrame
print(top_df.shape)
# Display the first 10 rows of the DataFrame
top_df.head(10)

(87, 7)


,_id,BusinessName,BusinessType,AddressLine1,scores.Hygiene,scores.Structural,scores.ConfidenceInManagement
0,679991f1773e8248a2ca0ac8,Abbi Wines,Retailers - other,12 - 14 Lakedale Road,0,0,0
1,679991f1773e8248a2ca0a36,Iceland,Retailers - supermarkets/hypermarkets,144 - 146 Plumstead High Street,0,5,5
2,679991f1773e8248a2ca0a46,Howe and Co Fish and Chips - Van 17,Mobile caterer,Restaurant And Premises 107A Plumstead High St...,0,0,0
3,679991f1773e8248a2ca0a63,Volunteer,Pub/bar/nightclub,130 - 132 Plumstead High Street,0,0,0
4,679991f1773e8248a2ca0a7b,Plumstead Manor Nursery,Caring Premises,Plumstead Manor School Old Mill Road,0,0,5
5,679991f1773e8248a2ca0a7e,Atlantic Fish Bar,Takeaway/sandwich shop,35 Lakedale Road,0,0,5
6,679991f1773e8248a2ca0a80,Lumbini Grocery Ltd T/A Al-Iman,Retailers - other,17 Lakedale Road,0,5,0
7,679991f1773e8248a2ca0a9e,Greggs,Takeaway/sandwich shop,6 Lakedale Road,0,0,0
8,679991f1773e8248a2ca0a9f,Tesco,Retailers - supermarkets/hypermarkets,18 Lakedale Road,0,5,0
9,679991f1773e8248a2ca0ac6,Dosa & Sambal Express,Restaurant/Cafe/Canteen,107A Plumstead High Street,0,0,5


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [27]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
match_query = {'$match': {'scores.Hygiene': 0
                         }
              }
# 2. Groups the matches by Local Authority
group_query = {'$group': {'_id': {
                                    'LocalAuthorityName': "$LocalAuthorityName"
                                },
                          'num_restaurants': { '$sum': 1 }, 
                         }
              }

# 3. Sorts the matches from highest to lowest
sort_values = {'$sort': { 'num_restaurants': -1
                        }
              }

pipeline = [match_query, group_query, sort_values]

# Run the aggregation pipeline
results = establishments.aggregate(pipeline)

# Print the number of documents in the result
results_df = list(results) 
print("Number of documents:", len(results_df))

# Print the number of documents in the result
print("Number of documents after aggregation:", len(results_df))

# Print the first 10 results
pprint(results_df[0:10])

Number of documents: 55
Number of documents after aggregation: 55
[{'_id': {'LocalAuthorityName': 'Thanet'}, 'num_restaurants': 1130},
 {'_id': {'LocalAuthorityName': 'Greenwich'}, 'num_restaurants': 882},
 {'_id': {'LocalAuthorityName': 'Maidstone'}, 'num_restaurants': 713},
 {'_id': {'LocalAuthorityName': 'Newham'}, 'num_restaurants': 711},
 {'_id': {'LocalAuthorityName': 'Swale'}, 'num_restaurants': 686},
 {'_id': {'LocalAuthorityName': 'Chelmsford'}, 'num_restaurants': 680},
 {'_id': {'LocalAuthorityName': 'Medway'}, 'num_restaurants': 672},
 {'_id': {'LocalAuthorityName': 'Bexley'}, 'num_restaurants': 607},
 {'_id': {'LocalAuthorityName': 'Southend-On-Sea'}, 'num_restaurants': 586},
 {'_id': {'LocalAuthorityName': 'Tendring'}, 'num_restaurants': 542}]


In [31]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(results_df)

# Display the number of rows in the DataFrame
print("Number of rows with a hygiene of 0:", len(df))

# Display the first 10 rows of the DataFrame
print("First 10 rows")
print(df.head(10))

Number of rows with a hygiene of 0: 55
First 10 rows
                                         _id  num_restaurants
0           {'LocalAuthorityName': 'Thanet'}             1130
1        {'LocalAuthorityName': 'Greenwich'}              882
2        {'LocalAuthorityName': 'Maidstone'}              713
3           {'LocalAuthorityName': 'Newham'}              711
4            {'LocalAuthorityName': 'Swale'}              686
5       {'LocalAuthorityName': 'Chelmsford'}              680
6           {'LocalAuthorityName': 'Medway'}              672
7           {'LocalAuthorityName': 'Bexley'}              607
8  {'LocalAuthorityName': 'Southend-On-Sea'}              586
9         {'LocalAuthorityName': 'Tendring'}              542
